In [1]:
import numpy as np
import pandas as pd
import datetime
import yfinance as yf
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# For importing universal scripts
import sys
import os
# Go up two levels from the subfolder
sys.path.append(os.path.abspath("../.."))
from indicators_returns import final_df #Universal script for indicator set and actuals

In [2]:
ticker = 'QQQ'
returns = [5, 10, 20, 30, 45, 60, 90]
lb = 20
df = final_df(ticker, returns, lb)
df = df.iloc[:-101].replace([np.inf, -np.inf], 0)

# Volatility, Zscore and Auto Correlation DF

In [10]:
from scipy.stats import skew, kurtosis, zscore

vol_zscore_stats = []

# Approximate trading days per year
years_to_days = {5: 1260, 10: 2520, 15: 3780}

for r in returns:
    df_test = df.copy()
    return_col = f'Return_{r}'
    indicator_cols = [col for col in df_test.columns 
                    if col not in ['Date', 'Close', 'High', 'Low', 'Volume']
                    and not col.startswith('Return')
                    and 'slope' not in col.lower()
    ]

    for col in indicator_cols:
        x_full = df_test[col].dropna()
        if len(x_full) < 400:  # need data for longest z-score window
            continue

        row = {'r': r, 'indicator': col}

        # Rolling volatility stats
        for w in [21, 42, 63]:
            row[f'avg_vol_{w}'] = df_test[col].rolling(window=w).std().mean()

        # Z-score stats by time horizon
        for y, lookback in years_to_days.items():
            x = x_full[-lookback:]
            if len(x) < 100:
                continue

            x_z = zscore(x)
            row[f'tight_zrange_pct_{y}y'] = ((x_z >= -0.1) & (x_z <= 0.1)).mean().round(4)
            row[f'skew_{y}y'] = skew(x_z)
            row[f'kurtosis_{y}y'] = kurtosis(x_z)

        # Autocorrelation stats (based on latest full series)
        for lag in [1, 5, 10]:
            if len(x_full) > lag:
                row[f'autocorr_lag{lag}'] = x_full.autocorr(lag=lag)

        vol_zscore_stats.append(row)

vol_z_df = pd.DataFrame(vol_zscore_stats)
vol_z_df.to_csv('indicator_dynamics.csv', index=False)